## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [16]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt

%matplotlib qt

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)#(54, 3)
#print(np.mgrid[0:9,0:6].T.reshape(-1,2))
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)


# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.
# Make a list of calibration images
images = glob.glob('../camera_cal/calibration*.jpg')
img=cv2.imread(images[0])
'''plt.subplot(1,2,1)
plt.imshow(img)
plt.subplot(1,2,2)
plt.imshow(img_undist)'''
# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)
        #cv2.drawChessboardCorners(img, (9,6), corners, ret)
        
       
        # Draw and display the corners
        #plt.subplot(4,5,fname.index(fname)+1)
        
        #cv2.imshow('img',img)
        #cv2.waitKey(600)
ret,mtx,dist,rvecs,tvecs=cv2.calibrateCamera(objpoints,imgpoints,img.shape[1::-1],None,None)
#cv2.destroyAllWindows()
def undistortion(img):
    return cv2.undistort(img,mtx,dist,None,mtx)

## And so on and so forth...

In [17]:
import matplotlib.image as mpimg
def thr_rgb(image,rthr=(0,255),gthr=(0,255),bthr=(0,255)):#此函数用于决定RGB中哪个通道最适合以及合适的阈值
    #gray=cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    R=image[:,:,0]
    G=image[:,:,1]
    B=image[:,:,2]
    binaryr = np.zeros_like(R)
    binaryr[(R > rthr[0]) & (R <= rthr[1])] = 1
    binaryg = np.zeros_like(R)
    binaryg[(G > gthr[0]) & (G <= gthr[1])] = 1
    binaryb = np.zeros_like(R)
    binaryb[(B > bthr[0]) & (B <= bthr[1])] = 1
    if(rthr[0]!=0 & rthr[1]!=255 ):
        print(1)
        return binaryr
    elif(gthr[0]!=0 &gthr[1]!=255):
        
        return binaryg
    else:
        return binaryb

#plt.imshow(image)

In [18]:
def thr_hls(image,hthr=(0,179),lthr=(0,255),sthr=(0,255)):
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    H=hls[:,:,0]
    L=hls[:,:,1]
    S=hls[:,:,2]
    binaryh = np.zeros_like(H)
    binaryh[(H > hthr[0]) & (H <= hthr[1])] = 1
    binaryl = np.zeros_like(H)
    binaryl[(L > lthr[0]) & (L <= lthr[1])] = 1
    binarys = np.zeros_like(H)
    binarys[(S > sthr[0]) & (S <= sthr[1])] = 1
    if(hthr[0]!=0 & hthr[1]!=179):
        return binaryh
    elif(lthr[0]!=0 & lthr[1]!=255):
        return binaryl
    else:
        return binarys

In [19]:
def thr_sobel(image,sobelx_thr=(0,255)):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    abs_sobelx = np.absolute(sobelx)
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    #plt.imshow(scaled_sobel,cmap='gray')
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sobelx_thr[0]) & (scaled_sobel <= sobelx_thr[1])] = 1
    return sxbinary

In [69]:
import os#此格用于查看8副图片各种threshold的效果
dic=os.listdir("../test_images/")
#plt.figure(figsize=(80,80),dpi=180)

#image=mpimg.imread("../test_images/"+dic[7])
#image=thr_sobel(image,sobelx_thr=(40,130))
#plt.imshow(image,cmap='gray')
for i in range(len(dic)):
    image=mpimg.imread("../test_images/"+dic[i])
    undistort=undistortion(image)
    image1=thr_rgb(undistort,rthr=(0,255),gthr=(180,255),bthr=(0,255))
    
    image2=thr_hls(undistort,hthr=(0,255),lthr=(0,255),sthr=(120,255))#120
    image3=thr_sobel(undistort,sobelx_thr=(30,130))#最小值大于40很多边缘检测不到
    #combine=image1&image2|image3
    color_img=255*np.dstack((image1,image2,image3))
    #warped=persp_trans(combine,src,dst)
    
    #pixelbelongtolane=lane_pixels(warped)
    #image_3d=fit_poly_3d(warped)
    result=detect(image)
    plt.subplot(2,4,i+1)
    #plt.imshow(warped,cmap='gray')
    #plt.plot(his_image)
    plt.imshow(color_img)

In [27]:
img2=mpimg.imread("../test_images/"+dic[1])
src=np.float32([[605,442],[675,442],[275,674],[1042,674]])
dst=np.float32([[300,0],[1000,0],[300,719],[1000,719]])

'''cv2.circle(img2,(605,442),10,(0,255,0),20)
cv2.circle(img2,(675,442),10,(0,255,0),20)
cv2.circle(img2,(1042,674),10,(0,255,0),20)
cv2.circle(img2,(275,674),10,(0,255,0),20)'''
#plt.imshow(img2)
def persp_trans(img,src,dst):
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, M, (1280,720))
    return warped

In [5]:
def hist(image):
    half_image=image[image.shape[0]//2:,:]
    histgram_image=np.sum(half_image,axis=0)
    return histgram_image

In [6]:
def lane_pixels(image):#利用sliding window寻找车道线
    his=hist(image)
    image_3d=np.dstack((image,image,image))
    mid=image.shape[1]//2
    lxbase=np.argmax(his[200:mid])+200
    rxbase=np.argmax(his[mid:-200])+mid
    numofwindow=10
    winheight=np.int(image.shape[0]//numofwindow)
    margin=100
    minpix=50
    nonzero = image.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    lx_current = lxbase
    rx_current = rxbase
    left_index=[]
    right_index=[]
    for i in range(numofwindow):
        win_y_low = image.shape[0] - (i+1)*winheight
        win_y_high = image.shape[0] - i*winheight
        win_xleft_low = lx_current - margin
        win_xleft_high = lx_current + margin
        win_xright_low = rx_current - margin
        win_xright_high = rx_current + margin
        
        cv2.rectangle(image_3d,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 4) 
        cv2.rectangle(image_3d,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 4) 
        
        inwin_left_index = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        inwin_right_index = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        left_index.append(inwin_left_index)
        right_index.append(inwin_right_index)
        
        if len(inwin_left_index) > minpix:
            lx_current = np.int(np.mean(nonzerox[inwin_left_index]))
        if len(inwin_right_index) > minpix:        
            rx_current = np.int(np.mean(nonzerox[inwin_right_index]))
        
    left_index = np.concatenate(left_index)
    right_index = np.concatenate(right_index)
    leftx = nonzerox[left_index]
    lefty = nonzeroy[left_index] 
    rightx = nonzerox[right_index]
    righty = nonzeroy[right_index]
        
    return leftx, lefty, rightx, righty, image_3d
    #return image_3d

In [56]:
def fit_poly_3d(image):#拟合并画车道线
    leftx, lefty, rightx, righty, image_3d = lane_pixels(image)
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    ploty = np.linspace(0, image.shape[0]-1, image.shape[0] )
  
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    image_3d[lefty, leftx] = [255, 0, 0]
    image_3d[righty, rightx] = [0, 0, 255]
    
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
   
    return image_3d,left_fit,right_fit,left_fitx,right_fitx,ploty
    

In [64]:
def get_curvature(undist,left_fitx,right_fitx,ploty):
    ym_per_pix = 30/720 
    xm_per_pix = 3.7/700
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
    y_eval = np.max(ploty*ym_per_pix)
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    car_center = undist.shape[1] / 2
    i = undist.shape[0]-1
    vehicle_position = (car_center - (right_fitx[i] + left_fitx[i])/2 )* xm_per_pix 
    vehicle_position=float('%.2f' % vehicle_position)
    return left_curverad,right_curverad,vehicle_position
    

In [70]:
def print_on_original(undist ,warped ,dst ,src ,left_fitx ,right_fitx ,ploty ,left_curverad ,right_curverad ,vehicle_position):
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    M = cv2.getPerspectiveTransform(dst, src)
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    if (vehicle_position > 0.3) | (vehicle_position < -0.3)  :
        cv2.fillPoly(color_warp, np.int_([pts]), (255,0, 0))
    else :
        cv2.fillPoly(color_warp, np.int_([pts]), (0,255,0))
    
    cv2.polylines(color_warp, np.int32([pts_left]), isClosed=False, color=(255,255,0), thickness=20)
    cv2.polylines(color_warp, np.int32([pts_right]), isClosed=False, color=(255,255,255), thickness=20)  
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, M, (img.shape[1], img.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    ### writting the curvature of the lane and vehicle position with respect to center   
    font = cv2.FONT_HERSHEY_SIMPLEX
    average=str(int((left_curverad+right_curverad)/2))
     
    result = cv2.putText(result, 'Radius of Curvature:  ' + average+'m'  , (30, 30), font, 0.8, (0, 255, 0), 2, cv2.LINE_AA) 
    if vehicle_position<=0:
        result = cv2.putText(result, 'Vehicle is '+str(-vehicle_position)+'m left of center' , (30, 60), font, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
    else:
        result = cv2.putText(result, 'Vehicle is '+str(vehicle_position)+'m right of center' , (30, 60), font, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
        

    return result
    
    
    
    
    

In [8]:
def fit_poly_last(img_shape, leftx, lefty, rightx, righty):##依据上一帧来拟合这一帧
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    return left_fitx, right_fitx, ploty
    #return image_3d

In [9]:
def near_search(image,left_fit,right_fit,ploty):#依据上一帧来检测这一帧
    margin=100
    nonzero = image.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    inwin_left_index = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    inwin_right_index = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    leftx = nonzerox[inwin_left_index]
    lefty = nonzeroy[inwin_left_index] 
    rightx = nonzerox[inwin_right_index]
    righty = nonzeroy[inwin_right_index]
    left_fitx, right_fitx, ploty = fit_poly_last(image.shape, leftx, lefty, rightx, righty)
    out_img = np.dstack((image, image, image))*255
    window_img = np.zeros_like(out_img)
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]
    
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin,ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin,ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    return result,left_fitx,right_fitx,ploty
    

In [47]:
#try it on single image,
'''timage = mpimg.imread("../test_images/"+dic[7])
tundistort=undistortion(timage)
timage1=thr_rgb(tundistort,rthr=(0,255),gthr=(180,255),bthr=(0,255))
   
timage2=thr_hls(tundistort,hthr=(0,255),lthr=(0,255),sthr=(120,255))
timage3=thr_sobel(tundistort,sobelx_thr=(38,130))#30最小值大于40很多边缘检测不到
tcombine=timage1&timage2|timage3   
twarped=persp_trans(tcombine,src,dst)
fit=fit_poly_3d(twarped)
curve=get_curvature(tundistort,fit[3] ,fit[4] ,fit[5])
result=print_on_original(tundistort ,twarped ,dst ,src ,fit[3] ,fit[4] ,fit[5] ,curve[0] ,curve[1] ,curve[2])

fitpoly=fit[0]

second_img=mpimg.imread("../test_images/"+dic[2])
stundistort=undistortion(second_img)
stimage1=thr_rgb(stundistort,rthr=(0,255),gthr=(180,255),bthr=(0,255))
stimage2=thr_hls(stundistort,hthr=(0,255),lthr=(0,255),sthr=(120,255))
stimage3=thr_sobel(stundistort,sobelx_thr=(30,130))#最小值大于40很多边缘检测不到
stcombine=stimage1&stimage2|stimage3
stwarped=persp_trans(stcombine)

search_near=near_search(stwarped,fit[1],fit[2],fit[3])
plt.imshow(fit[0])'''

In [49]:
def detect(image):
    undistort=undistortion(image)
    image1=thr_rgb(undistort,rthr=(0,255),gthr=(180,255),bthr=(0,255))
    image2=thr_hls(undistort,hthr=(0,255),lthr=(0,255),sthr=(120,255))#120
    image3=thr_sobel(undistort,sobelx_thr=(30,130))#最小值大于40很多边缘检测不到
    combine=image1&image2|image3
    warped=persp_trans(combine,src,dst)
    fit=fit_poly_3d(warped)
    curve=get_curvature(undistort,fit[3] ,fit[4] ,fit[5])
    result=print_on_original(undistort ,warped ,dst ,src ,fit[3] ,fit[4] ,fit[5] ,curve[0] ,curve[1] ,curve[2])
    return result


In [50]:
!pip install moviepy
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [71]:
white_output = 'project_video_output.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(detect) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video project_video_output.mp4
[MoviePy] Writing video project_video_output.mp4


100%|█████████████████████████████████████▉| 1260/1261 [04:21<00:00,  5.27it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_output.mp4 

Wall time: 4min 23s


In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))